<a href="https://colab.research.google.com/github/sunnysavita10/Generative-AI-Indepth-Basic-to-Advance/blob/main/basic_retrieval_and_contextual_compression_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/langchain-ai/langchain/tree/master/libs/langchain/langchain/retrievers/document_compressors

https://blog.langchain.dev/improving-document-retrieval-with-contextual-compression/

In [ ]:
!pip install langchain_community

In [ ]:
!pip install langchain_openai

In [ ]:
#facebook ai similarity search
!pip install faiss-cpu

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

In [ ]:
documents = TextLoader("/content/state_of_the_union.txt").load()

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)

In [ ]:
texts = text_splitter.split_documents(documents)

In [ ]:
texts

In [ ]:
from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
retriever = FAISS.from_documents(texts, OpenAIEmbeddings()).as_retriever()

In [ ]:
docs = retriever.invoke("What did the president say about Ketanji Brown Jackson")

In [ ]:
# Helper function for printing docs

def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [ ]:
pretty_print_docs(docs)

In [ ]:
docs2 = retriever.invoke("What were the top three priorities outlined in the most recent State of the Union address?")

In [ ]:
pretty_print_docs(docs2)

In [ ]:
docs3 = retriever.invoke("How did the President propose to tackle the issue of climate change?")


In [ ]:
pretty_print_docs(docs3)

In [ ]:
from langchain_openai import OpenAI

In [ ]:
llm=OpenAI(temperature=0)

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

In [ ]:
query="What were the top three priorities outlined in the most recent State of the Union address?"

In [ ]:
chain.invoke(query)

In [ ]:
print(chain.invoke(query)['result'])

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_openai import OpenAI

In [ ]:
compressor = LLMChainExtractor.from_llm(llm)

In [ ]:
compression_retriever=ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever)

In [ ]:
compressed_docs = compression_retriever.invoke("What did the president say about Ketanji Jackson Brown")

In [ ]:
compressed_docs

In [ ]:
compressed_docs = compression_retriever.invoke("What were the top three priorities outlined in the most recent State of the Union address?")

In [ ]:
compressed_docs

In [ ]:
pretty_print_docs(compressed_docs)

In [ ]:
compressed_docs2 = compression_retriever.invoke("How did the President propose to tackle the issue of climate change?")

In [ ]:
pretty_print_docs(compressed_docs2)

In [ ]:
from langchain.retrievers.document_compressors import LLMChainFilter

In [ ]:
filter = LLMChainFilter.from_llm(llm)

In [ ]:
compression_retriever2 = ContextualCompressionRetriever(base_compressor=filter, base_retriever=retriever)

In [ ]:
compressed_docs3 = compression_retriever2.invoke("What were the top three priorities outlined in the most recent State of the Union address?")

In [ ]:
pretty_print_docs(compressed_docs3)

In [ ]:
original_contexts_len = len("\n\n".join([d.page_content for i, d in enumerate(docs2)]))

In [ ]:
original_contexts_len

In [ ]:
compressed_contexts_len = len("\n\n".join([d.page_content for i, d in enumerate(compressed_docs)]))

In [ ]:
compressed_contexts_len

In [ ]:
print("Original context length:", original_contexts_len)

In [ ]:
print("Compressed context length:", compressed_contexts_len)

In [ ]:
print("Compressed Ratio:", f"{original_contexts_len/(compressed_contexts_len + 1e-5):.2f}x")

In [ ]:
from langchain.retrievers.document_compressors import EmbeddingsFilter

In [ ]:
from langchain_openai import OpenAIEmbeddings

In [ ]:
embeddings = OpenAIEmbeddings()

In [ ]:
embeddings_filter = EmbeddingsFilter(embeddings=embeddings)

In [ ]:
compression_retriever3 = ContextualCompressionRetriever(base_compressor=embeddings_filter, base_retriever=retriever)

In [ ]:
compressed_docs4 = compression_retriever3.invoke("What were the top three priorities outlined in the most recent State of the Union address?")

In [ ]:
pretty_print_docs(compressed_docs4)

In [ ]:
print("Original context length:", original_contexts_len)

In [ ]:
compressed_contexts_len = len("\n\n".join([d.page_content for i, d in enumerate(compressed_docs)]))

In [ ]:
print("Compressed context length:", compressed_contexts_len)

In [ ]:
print("Compressed Ratio:", f"{original_contexts_len/(compressed_contexts_len + 1e-5):.2f}x")

In [ ]:
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain_text_splitters import CharacterTextSplitter


In [ ]:
splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0, separator=". ")

In [ ]:
redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings)

In [ ]:
relevant_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.76)

In [ ]:
pipeline_compressor = DocumentCompressorPipeline(transformers=[splitter, redundant_filter, relevant_filter])

In [ ]:
compression_retriever = ContextualCompressionRetriever(base_compressor=pipeline_compressor, base_retriever=retriever)

In [ ]:
compressed_docs = compression_retriever.invoke("What were the top three priorities outlined in the most recent State of the Union address?")

In [ ]:
pretty_print_docs(compressed_docs)

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0)

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
chain = RetrievalQA.from_chain_type(llm=llm, retriever=compression_retriever)

In [ ]:
query="What were the top three priorities outlined in the most recent State of the Union address?"

In [ ]:
chain.invoke(query)

In [ ]:
print(chain.invoke(query)['result'])

The top three priorities outlined in the most recent State of the Union address were:

1. Beating the opioid epidemic by increasing funding for prevention, treatment, harm reduction, and recovery.
2. Strengthening infrastructure and innovation in America to improve transportation and create more jobs.
3. Promoting domestic production and reducing reliance on foreign supply chains to boost the economy and create more opportunities for Americans.